Cuardeno para procesar la data cruda del saber tyt de DataICFES

### Constantes y Librerias 

In [45]:
import os
import glob
import pandas as pd
from unidecode import unidecode
import numpy as np
import csv
#Archivo con constantes
import constants

### Lectura informacion cruda del saber 11

In [46]:
#Directorios con la datos del saber tyt
data_dir_base_sbtyt = "../data/SABERTYT_raw/"

#lista con la ruta de los archivos 
csv_files_sabertyt = glob.glob(os.path.join(data_dir_base_sbtyt, "*.txt"))

In [47]:
#Variables a seleccionar del saber tyt 
# Convertimos a set para búsquedas más rápidas
cols_set = set(constants.cols_saber_tyt_lower)

In [ ]:
# Lista donde se irán acumulando los DataFrames leídos
data_list = []

# Iteramos por cada archivo y su separador asociado
for file_name, sep in constants.nombre_archivos_sabertyt.items():
    file_path = os.path.join(data_dir_base_sbtyt, file_name)

    # Elegimos el motor según el separador
    engine = "python" if sep == "¬" else "c"

    # Diccionario con argumentos comunes y condicionales

    read_csv_args = {
        "sep": sep,
        "usecols": lambda c: c.lower() in cols_set, # Solo cargar columnas cuyo nombre (en minúscula) esté en la lista deseada
        "engine": engine,
        **(
            {"quotechar": None, "quoting": csv.QUOTE_NONE, "dtype": str}  # Si el separador es '¬', desactivar manejo de comillas y forzar lectura como texto
            if sep == "¬" else
            {"low_memory": False} # Para otros separadores, mejorar manejo de tipos con low_memory=False
        )
    }
    try:
        # Leer el archivo con los argumentos definidos
        df = pd.read_csv(file_path, **read_csv_args)

        # Convertir todos los nombres de columnas a minúsculas
        df.columns = df.columns.str.lower()

        # Verificar si faltan columnas esperadas y avisar
        faltantes = cols_set - set(df.columns)
        if faltantes:
            print(f"Atención: en {file_name} faltan columnas: {faltantes}")

        # Agregar el DataFrame leído a la lista
        data_list.append(df)

    except Exception as err:
        # Reportar errores durante la lectura
        print(f"Error leyendo {file_name} con separador '{sep}': {err}")

# Combinar todos los DataFrames en uno solo
base_sbtyt = pd.concat(data_list, ignore_index=True)


### Ajustes de formato 

Tipado de variables

In [49]:
#Parse las columnas a los tipos de variables adecuados
for col, dtype in constants.dtype_mapping_sabertyt.items():
    if col in base_sbtyt.columns:
        try:
            base_sbtyt[col] = base_sbtyt[col].astype(dtype)
        except Exception as e:
            print(f"No se pudo convertir la columna {col} a {dtype}: {e}")


cols_numericas = [
    "periodo",
    "inst_cod_institucion",
    "estu_snies_prgmacademico",
    "estu_prgm_codmunicipio"
]        
#parse a numerico las variables numericas
for col in cols_numericas:
    base_sbtyt[col] = pd.to_numeric(base_sbtyt[col], errors='coerce').astype('Int64')

Ajuste de variables

In [50]:
#Ajustar los periodos para agregarlos anualmente
base_sbtyt['año_presentacion'] = base_sbtyt['periodo'].apply(lambda x: x//10)

### Filtrar por Bogota-Region

In [51]:
#Codigos y nombre Municipios Bogotá Region
municipios_bogota_region = pd.read_csv("../data/Municipios_cleaned/municipios.csv")

#Quedarse con las observaciones del saber tyt donde el programa educativo esta ubicado en Bogota-Region 
base_sbtyt = base_sbtyt[base_sbtyt["estu_prgm_codmunicipio"].isin(municipios_bogota_region["codigo_dane_municipio"])]


In [52]:
base_sbtyt = base_sbtyt.sort_values(by="periodo")
base_sbtyt = base_sbtyt.reset_index(drop=True)

print(f"Dimensiones: {base_sbtyt.shape}")
base_sbtyt.head()

Dimensiones: (185818, 14)


,periodo,estu_consecutivo,inst_cod_institucion,inst_nombre_institucion,estu_prgm_academico,estu_snies_prgmacademico,estu_prgm_codmunicipio,estu_prgm_municipio,estu_nivel_prgm_academico,estu_nucleo_pregrado,mod_razona_cuantitat_punt,mod_lectura_critica_punt,punt_global,año_presentacion
1523,20201,EK202010003355,1207,UNIVERSIDAD DEL TOLIMA-IBAGUE,TECNOLOGIA EN REGENCIA DE FARMACIA,53371,11001,BOGOTÁ D.C.,TECNOLOGÍA,"BIOLOGÍA, MICROBIOLOGÍA Y AFINES",128.0,116.0,130.0,2020
1524,20201,EK202010002387,1207,UNIVERSIDAD DEL TOLIMA-IBAGUE,TECNOLOGIA EN REGENCIA DE FARMACIA,53371,11001,BOGOTÁ D.C.,TECNOLOGÍA,"BIOLOGÍA, MICROBIOLOGÍA Y AFINES",115.0,133.0,94.0,2020
1525,20201,EK202010032494,1207,UNIVERSIDAD DEL TOLIMA-IBAGUE,TECNOLOGIA EN GESTION DE BASES DE DATOS,102261,11001,BOGOTÁ D.C.,TECNOLOGÍA,"INGENIERÍA DE SISTEMAS, TELEMÁTICA Y AFINES",136.0,136.0,141.0,2020
1526,20201,EK202010083631,1207,UNIVERSIDAD DEL TOLIMA-IBAGUE,TECNOLOGIA EN REGENCIA DE FARMACIA,53371,11001,BOGOTÁ D.C.,TECNOLOGÍA,"BIOLOGÍA, MICROBIOLOGÍA Y AFINES",103.0,97.0,104.0,2020
1527,20201,EK202010031002,1207,UNIVERSIDAD DEL TOLIMA-IBAGUE,TECNOLOGIA EN GESTION DE BASES DE DATOS,102261,11001,BOGOTÁ D.C.,TECNOLOGÍA,"INGENIERÍA DE SISTEMAS, TELEMÁTICA Y AFINES",107.0,107.0,102.0,2020


### Estadisticas descriptivas

In [61]:
num_programas_unicos  = base_sbtyt['estu_snies_prgmacademico'].nunique()
num_ies_unicas  = base_sbtyt['inst_cod_institucion'].nunique()
num_estudiantes_unicos = base_sbtyt['estu_consecutivo'].nunique()
periodos_unicos = base_sbtyt['periodo'].unique()
años_unicos = base_sbtyt['año_presentacion'].unique()

In [62]:
print(
    f"Numero programas unicos: {num_programas_unicos} \n" 
    f"Numero IES unicas: {num_ies_unicas}\n"
    f"Numero estudiantes unicos: {num_estudiantes_unicos} de {base_sbtyt.shape[0]}\n"
    f"Periodos presentacion saber TYT:{periodos_unicos}\n"
    f"Años presentacion saber TYT:{años_unicos}\n"
    )

Numero programas unicos: 901 
Numero IES unicas: 89
Numero estudiantes unicos: 185818 de 185818
Periodos presentacion saber TYT:<IntegerArray>
[20201, 20203, 20202, 20211, 20212, 20213, 20221, 20223, 20224, 20226, 20232,
 20231, 20234, 20233]
Length: 14, dtype: Int64
Años presentacion saber TYT:[2020 2021 2022 2023]



### Guardar dataframe

In [ ]:
#Guardar el dataframe como un archivo csv en la carpeta SABERTyT_cleaned
base_sbtyt.to_csv("../data/SABERTYT_cleaned/base_sbtyt.csv", index=False)